In [2]:
import tabula
from tabula import read_pdf
import pandas as pd
import numpy as np
import os
import requests, zipfile, io
import zipfile
import sys
sys._enablelegacywindowsfsencoding()
import re
import shutil, os
from google_drive_downloader import GoogleDriveDownloader as gdd
from datetime import date
from datetime import datetime
import arrow
import pendulum
import urllib.request

In [3]:
url = "https://www.dropbox.com/sh/4dgzsyo2q4qbpw6/AABW921AbFtQsPvnL-7ayXGHa/Municipalities/municip_totalCIEISP.dta?dl=1"  # dl=1 is important

u = urllib.request.urlopen(url)
data = u.read()
u.close()
 
with open('municip_totalCIEISP.dta', "wb") as f :
    f.write(data)
    

mpal_hist = pd.read_stata('municip_totalCIEISP.dta',convert_categoricals=True)



url = "https://www.dropbox.com/sh/4dgzsyo2q4qbpw6/AAC5LKOlal7sqQ30xq6u1xiEa/Municipalities/Files/mastermpal.dta?dl=1"  # dl=1 is important
u = urllib.request.urlopen(url)
data = u.read()
u.close()
 
with open('mastermpal.dta', "wb") as f :
    f.write(data)
    

mpal = pd.read_stata('mastermpal.dta',convert_categoricals=False)


url = "https://www.dropbox.com/sh/4dgzsyo2q4qbpw6/AACjwmI52YTURzfHcQxVfhSCa/Municipalities/Files/mun_code.dta?dl=1"  # dl=1 is important

u = urllib.request.urlopen(url)
data = u.read()
u.close()
 
with open('mun_code.dta', "wb") as f :
    f.write(data)
mun = pd.read_stata('mun_code.dta',convert_categoricals=False)
    

mpal_mun = pd.merge(mpal,mun)

crimes = pd.read_csv('inputs\crime_code.csv', encoding='latin1')

mpal_delitos = pd.merge(mpal_mun,crimes)
EDO_mun = mpal_delitos[mpal_delitos.state_code == 15]
to_drop = ['Total de delitos','Feminicidio con arma de fuego', 'Lesiones dolosas con arma de fuego','Homicidio doloso con arma de fuego', 'Homicidio doloso sin Fem','Homicidio doloso con arma de fuego sin Fem','Robo en espacio o vía pública','Robo en transporte','Violencia familiar','Lesiones dolosas']
EDO_mun=EDO_mun[~EDO_mun['Delitos'].isin(to_drop)]




In [7]:
mpal_mun.to_csv(r'C:\Users\Angel ONC\Desktop\Curso\mun_pob.csv')

In [3]:
absolutos_historico_path = r'~\Google Drive\DFSC 2019\Estado de México\inputs\edomex_historico_absolutos.csv'

In [4]:
absolutos_historico= pd.read_csv(absolutos_historico_path, engine='python')

In [5]:
absolutos_historico = absolutos_historico.set_index('mun_code')

In [6]:
mun_pop_hist = pd.read_csv('inputs/munpophist.csv', index_col='mun_code')

In [7]:
pob = pd.read_csv('inputs/distritos_judiciales.csv', engine='python')

In [8]:
pob.head()

,CVE_ENT,CVE_MUN,mun_code,NOMGEO,num,distrito
0,15,9,15009,Amecameca,I,Chalco
1,15,15,15015,Atlautla,I,Chalco
2,15,17,15017,Ayapango,I,Chalco
3,15,25,15025,Chalco,I,Chalco
4,15,22,15022,Cocotitlán,I,Chalco


In [9]:
pob = pob.set_index('mun_code')

In [10]:
pob_reg_hist  = pd.concat([pob, mun_pop_hist], axis=1)


In [11]:
pob_reg_hist_grouped = pob_reg_hist.groupby('distrito').sum()

In [12]:

abs_hist_reg = pd.merge(pob, absolutos_historico, left_index=True, right_index=True, how='outer')

In [13]:
abs_reg_hist_grouped = abs_hist_reg.groupby(['distrito','Delito']).sum()

In [14]:
rates_reg_hist_grouped =  (abs_reg_hist_grouped/pob_reg_hist_grouped)*100000

In [16]:
rates_reg_hist_grouped =rates_reg_hist_grouped.drop(['CVE_ENT', 'CVE_MUN', 'Unnamed: 0', 'state_code'], axis=1)

In [17]:
rates_reg_hist_grouped['S1_2011'] = rates_reg_hist_grouped.ene2011 + rates_reg_hist_grouped.feb2011 + rates_reg_hist_grouped.mar2011 + rates_reg_hist_grouped.abr2011+  rates_reg_hist_grouped.may2011 + rates_reg_hist_grouped.jun2011
rates_reg_hist_grouped['S2_2011'] = rates_reg_hist_grouped.jul2011 + rates_reg_hist_grouped.ago2011 + rates_reg_hist_grouped.sep2011 + rates_reg_hist_grouped.oct2011+  rates_reg_hist_grouped.nov2011 + rates_reg_hist_grouped.dic2011
rates_reg_hist_grouped['C1_2011'] = rates_reg_hist_grouped.ene2011 + rates_reg_hist_grouped.feb2011 + rates_reg_hist_grouped.mar2011 + rates_reg_hist_grouped.abr2011
rates_reg_hist_grouped['C2_2011'] = rates_reg_hist_grouped.may2011 + rates_reg_hist_grouped.jun2011+ rates_reg_hist_grouped.jul2011 + rates_reg_hist_grouped.ago2011
rates_reg_hist_grouped['C3_2011'] = rates_reg_hist_grouped.sep2011 + rates_reg_hist_grouped.oct2011+  rates_reg_hist_grouped.nov2011 + rates_reg_hist_grouped.dic2011
rates_reg_hist_grouped['T1_2011'] = rates_reg_hist_grouped.ene2011 + rates_reg_hist_grouped.feb2011 + rates_reg_hist_grouped.mar2011
rates_reg_hist_grouped['T2_2011'] = rates_reg_hist_grouped.abr2011+  rates_reg_hist_grouped.may2011 + rates_reg_hist_grouped.jun2011
rates_reg_hist_grouped['T3_2011'] = rates_reg_hist_grouped.jul2011 + rates_reg_hist_grouped.ago2011 + rates_reg_hist_grouped.sep2011
rates_reg_hist_grouped['T4_2011'] = rates_reg_hist_grouped.oct2011+  rates_reg_hist_grouped.nov2011 + rates_reg_hist_grouped.dic2011
rates_reg_hist_grouped['B1_2011'] = rates_reg_hist_grouped.ene2011 + rates_reg_hist_grouped.feb2011 
rates_reg_hist_grouped['B2_2011'] = rates_reg_hist_grouped.mar2011 + rates_reg_hist_grouped.abr2011
rates_reg_hist_grouped['B3_2011'] = rates_reg_hist_grouped.may2011 + rates_reg_hist_grouped.jun2011
rates_reg_hist_grouped['B4_2011'] = rates_reg_hist_grouped.jul2011 + rates_reg_hist_grouped.ago2011
rates_reg_hist_grouped['B5_2011'] = rates_reg_hist_grouped.sep2011 + rates_reg_hist_grouped.oct2011
rates_reg_hist_grouped['B6_2011'] = rates_reg_hist_grouped.nov2011 + rates_reg_hist_grouped.dic2011
rates_reg_hist_grouped['A_2011'] = rates_reg_hist_grouped.ene2011 + rates_reg_hist_grouped.feb2011 + rates_reg_hist_grouped.mar2011 + rates_reg_hist_grouped.abr2011+  rates_reg_hist_grouped.may2011 + rates_reg_hist_grouped.jun2011+ rates_reg_hist_grouped.jul2011 + rates_reg_hist_grouped.ago2011 + rates_reg_hist_grouped.sep2011 + rates_reg_hist_grouped.oct2011+  rates_reg_hist_grouped.nov2011 + rates_reg_hist_grouped.dic2011

In [18]:
rates_reg_hist_grouped['S1_2012'] = rates_reg_hist_grouped.ene2012 + rates_reg_hist_grouped.feb2012 + rates_reg_hist_grouped.mar2012 + rates_reg_hist_grouped.abr2012+  rates_reg_hist_grouped.may2012 + rates_reg_hist_grouped.jun2012
rates_reg_hist_grouped['S2_2012'] = rates_reg_hist_grouped.jul2012 + rates_reg_hist_grouped.ago2012 + rates_reg_hist_grouped.sep2012 + rates_reg_hist_grouped.oct2012+  rates_reg_hist_grouped.nov2012 + rates_reg_hist_grouped.dic2012
rates_reg_hist_grouped['C1_2012'] = rates_reg_hist_grouped.ene2012 + rates_reg_hist_grouped.feb2012 + rates_reg_hist_grouped.mar2012 + rates_reg_hist_grouped.abr2012
rates_reg_hist_grouped['C2_2012'] = rates_reg_hist_grouped.may2012 + rates_reg_hist_grouped.jun2012+ rates_reg_hist_grouped.jul2012 + rates_reg_hist_grouped.ago2012
rates_reg_hist_grouped['C3_2012'] = rates_reg_hist_grouped.sep2012 + rates_reg_hist_grouped.oct2012+  rates_reg_hist_grouped.nov2012 + rates_reg_hist_grouped.dic2012
rates_reg_hist_grouped['T1_2012'] = rates_reg_hist_grouped.ene2012 + rates_reg_hist_grouped.feb2012 + rates_reg_hist_grouped.mar2012
rates_reg_hist_grouped['T2_2012'] = rates_reg_hist_grouped.abr2012+  rates_reg_hist_grouped.may2012 + rates_reg_hist_grouped.jun2012
rates_reg_hist_grouped['T3_2012'] = rates_reg_hist_grouped.jul2012 + rates_reg_hist_grouped.ago2012 + rates_reg_hist_grouped.sep2012
rates_reg_hist_grouped['T4_2012'] = rates_reg_hist_grouped.oct2012+  rates_reg_hist_grouped.nov2012 + rates_reg_hist_grouped.dic2012
rates_reg_hist_grouped['B1_2012'] = rates_reg_hist_grouped.ene2012 + rates_reg_hist_grouped.feb2012 
rates_reg_hist_grouped['B2_2012'] = rates_reg_hist_grouped.mar2012 + rates_reg_hist_grouped.abr2012
rates_reg_hist_grouped['B3_2012'] = rates_reg_hist_grouped.may2012 + rates_reg_hist_grouped.jun2012
rates_reg_hist_grouped['B4_2012'] = rates_reg_hist_grouped.jul2012 + rates_reg_hist_grouped.ago2012
rates_reg_hist_grouped['B5_2012'] = rates_reg_hist_grouped.sep2012 + rates_reg_hist_grouped.oct2012
rates_reg_hist_grouped['B6_2012'] = rates_reg_hist_grouped.nov2012 + rates_reg_hist_grouped.dic2012
rates_reg_hist_grouped['A_2012'] = rates_reg_hist_grouped.ene2012 + rates_reg_hist_grouped.feb2012 + rates_reg_hist_grouped.mar2012 + rates_reg_hist_grouped.abr2012+  rates_reg_hist_grouped.may2012 + rates_reg_hist_grouped.jun2012+ rates_reg_hist_grouped.jul2012 + rates_reg_hist_grouped.ago2012 + rates_reg_hist_grouped.sep2012 + rates_reg_hist_grouped.oct2012+  rates_reg_hist_grouped.nov2012 + rates_reg_hist_grouped.dic2012

In [19]:
rates_reg_hist_grouped['S1_2013'] = rates_reg_hist_grouped.ene2013 + rates_reg_hist_grouped.feb2013 + rates_reg_hist_grouped.mar2013 + rates_reg_hist_grouped.abr2013+  rates_reg_hist_grouped.may2013 + rates_reg_hist_grouped.jun2013
rates_reg_hist_grouped['S2_2013'] = rates_reg_hist_grouped.jul2013 + rates_reg_hist_grouped.ago2013 + rates_reg_hist_grouped.sep2013 + rates_reg_hist_grouped.oct2013+  rates_reg_hist_grouped.nov2013 + rates_reg_hist_grouped.dic2013
rates_reg_hist_grouped['C1_2013'] = rates_reg_hist_grouped.ene2013 + rates_reg_hist_grouped.feb2013 + rates_reg_hist_grouped.mar2013 + rates_reg_hist_grouped.abr2013
rates_reg_hist_grouped['C2_2013'] = rates_reg_hist_grouped.may2013 + rates_reg_hist_grouped.jun2013+ rates_reg_hist_grouped.jul2013 + rates_reg_hist_grouped.ago2013
rates_reg_hist_grouped['C3_2013'] = rates_reg_hist_grouped.sep2013 + rates_reg_hist_grouped.oct2013+  rates_reg_hist_grouped.nov2013 + rates_reg_hist_grouped.dic2013
rates_reg_hist_grouped['T1_2013'] = rates_reg_hist_grouped.ene2013 + rates_reg_hist_grouped.feb2013 + rates_reg_hist_grouped.mar2013
rates_reg_hist_grouped['T2_2013'] = rates_reg_hist_grouped.abr2013+  rates_reg_hist_grouped.may2013 + rates_reg_hist_grouped.jun2013
rates_reg_hist_grouped['T3_2013'] = rates_reg_hist_grouped.jul2013 + rates_reg_hist_grouped.ago2013 + rates_reg_hist_grouped.sep2013
rates_reg_hist_grouped['T4_2013'] = rates_reg_hist_grouped.oct2013+  rates_reg_hist_grouped.nov2013 + rates_reg_hist_grouped.dic2013
rates_reg_hist_grouped['B1_2013'] = rates_reg_hist_grouped.ene2013 + rates_reg_hist_grouped.feb2013 
rates_reg_hist_grouped['B2_2013'] = rates_reg_hist_grouped.mar2013 + rates_reg_hist_grouped.abr2013
rates_reg_hist_grouped['B3_2013'] = rates_reg_hist_grouped.may2013 + rates_reg_hist_grouped.jun2013
rates_reg_hist_grouped['B4_2013'] = rates_reg_hist_grouped.jul2013 + rates_reg_hist_grouped.ago2013
rates_reg_hist_grouped['B5_2013'] = rates_reg_hist_grouped.sep2013 + rates_reg_hist_grouped.oct2013
rates_reg_hist_grouped['B6_2013'] = rates_reg_hist_grouped.nov2013 + rates_reg_hist_grouped.dic2013
rates_reg_hist_grouped['A_2013'] = rates_reg_hist_grouped.ene2013 + rates_reg_hist_grouped.feb2013 + rates_reg_hist_grouped.mar2013 + rates_reg_hist_grouped.abr2013+  rates_reg_hist_grouped.may2013 + rates_reg_hist_grouped.jun2013+ rates_reg_hist_grouped.jul2013 + rates_reg_hist_grouped.ago2013 + rates_reg_hist_grouped.sep2013 + rates_reg_hist_grouped.oct2013+  rates_reg_hist_grouped.nov2013 + rates_reg_hist_grouped.dic2013

In [20]:
rates_reg_hist_grouped['S1_2014'] = rates_reg_hist_grouped.ene2014 + rates_reg_hist_grouped.feb2014 + rates_reg_hist_grouped.mar2014 + rates_reg_hist_grouped.abr2014+  rates_reg_hist_grouped.may2014 + rates_reg_hist_grouped.jun2014
rates_reg_hist_grouped['S2_2014'] = rates_reg_hist_grouped.jul2014 + rates_reg_hist_grouped.ago2014 + rates_reg_hist_grouped.sep2014 + rates_reg_hist_grouped.oct2014+  rates_reg_hist_grouped.nov2014 + rates_reg_hist_grouped.dic2014
rates_reg_hist_grouped['C1_2014'] = rates_reg_hist_grouped.ene2014 + rates_reg_hist_grouped.feb2014 + rates_reg_hist_grouped.mar2014 + rates_reg_hist_grouped.abr2014
rates_reg_hist_grouped['C2_2014'] = rates_reg_hist_grouped.may2014 + rates_reg_hist_grouped.jun2014+ rates_reg_hist_grouped.jul2014 + rates_reg_hist_grouped.ago2014
rates_reg_hist_grouped['C3_2014'] = rates_reg_hist_grouped.sep2014 + rates_reg_hist_grouped.oct2014+  rates_reg_hist_grouped.nov2014 + rates_reg_hist_grouped.dic2014
rates_reg_hist_grouped['T1_2014'] = rates_reg_hist_grouped.ene2014 + rates_reg_hist_grouped.feb2014 + rates_reg_hist_grouped.mar2014
rates_reg_hist_grouped['T2_2014'] = rates_reg_hist_grouped.abr2014+  rates_reg_hist_grouped.may2014 + rates_reg_hist_grouped.jun2014
rates_reg_hist_grouped['T3_2014'] = rates_reg_hist_grouped.jul2014 + rates_reg_hist_grouped.ago2014 + rates_reg_hist_grouped.sep2014
rates_reg_hist_grouped['T4_2014'] = rates_reg_hist_grouped.oct2014+  rates_reg_hist_grouped.nov2014 + rates_reg_hist_grouped.dic2014
rates_reg_hist_grouped['B1_2014'] = rates_reg_hist_grouped.ene2014 + rates_reg_hist_grouped.feb2014 
rates_reg_hist_grouped['B2_2014'] = rates_reg_hist_grouped.mar2014 + rates_reg_hist_grouped.abr2014
rates_reg_hist_grouped['B3_2014'] = rates_reg_hist_grouped.may2014 + rates_reg_hist_grouped.jun2014
rates_reg_hist_grouped['B4_2014'] = rates_reg_hist_grouped.jul2014 + rates_reg_hist_grouped.ago2014
rates_reg_hist_grouped['B5_2014'] = rates_reg_hist_grouped.sep2014 + rates_reg_hist_grouped.oct2014
rates_reg_hist_grouped['B6_2014'] = rates_reg_hist_grouped.nov2014 + rates_reg_hist_grouped.dic2014
rates_reg_hist_grouped['A_2014'] = rates_reg_hist_grouped.ene2014 + rates_reg_hist_grouped.feb2014 + rates_reg_hist_grouped.mar2014 + rates_reg_hist_grouped.abr2014+  rates_reg_hist_grouped.may2014 + rates_reg_hist_grouped.jun2014+ rates_reg_hist_grouped.jul2014 + rates_reg_hist_grouped.ago2014 + rates_reg_hist_grouped.sep2014 + rates_reg_hist_grouped.oct2014+  rates_reg_hist_grouped.nov2014 + rates_reg_hist_grouped.dic2014

In [21]:
VF = 'S1_2019'
VI = 'S1_2018'
VAI = 'S2_2018'

In [22]:
crimes = {
    'Extorsión':'5',
    'Homicidio culposo':'3',
    'Robo a transeúnte total':'10',
    'Narcomenudeo':'13',
    'Robo de vehículo':'7',
    'Robo con violencia':'6',
    'Violación':'11',
    'Feminicidio':'2',
    'Robo a casa habitación':'8',
    'Secuestro':'4',
    'Homicidio doloso':'1',
    'Robo de negocio':'9',
    'Trata de personas':'12'
    
}
EDO_mun['ordendelitos'] = EDO_mun.Delitos.replace(crimes, regex=True)
EDO_mun['ordendelitos'] = EDO_mun['ordendelitos'].apply(np.int64)
EDO_mun['orden'] = EDO_mun.Delitos.replace(crimes, regex=True)
EDO_mun['orden'] = EDO_mun['ordendelitos'].apply(np.int64)
EDO_mun = EDO_mun.set_index('orden')

In [23]:
EDO_mun_t = EDO_mun
EDO_mun = EDO_mun.loc[:, ~EDO_mun.columns.str.contains('t_')]


In [24]:
EDO_mun = EDO_mun.set_index('mun_code')

In [25]:
EDO_mun =EDO_mun.join(pob, how='outer')
EDO_mun_t = EDO_mun_t.set_index('mun_code')
EDO_mun_t =EDO_mun_t.join(pob, how='outer')

In [26]:
EDO_mun_t.to_csv('inputs/edomexfull.csv')

In [27]:
edo_mex = EDO_mun.groupby(['num', 'distrito','ordendelitos','Delitos']).sum()

In [28]:
edo_mex = edo_mex.drop(['state_code','t12015','t22015','t32015','t42015','t12016','t22016','t32016','t42016','t12017','t22017','t32017','t42017','t12018','t22018','t32018','t42018','t12019','t22019','t32019','t42019','c12015','c22015','c32015','c12016','c22016','c32016','c12017','c22017','c32017','c12018','c22018','c32018','c12019','c22019','c32019','s12015','s22015','s12016','s22016','s12017','s22017','s12018','s22018','s12019','s22019','a2015','a2016', 'a2017','a2018','a2019','crime','mun_codelabel','CVE_ENT','CVE_MUN'], axis=1)

In [29]:
edo_mex.to_csv('tmp/xout.csv')

In [30]:
edo_mex = pd.read_csv('tmp/xout.csv')
edo_mex = edo_mex.set_index('num','region')

In [31]:
pob_reg = pd.read_csv('inputs/distrito_juridico_edomex_pob.csv', engine='python')
pob_reg = pob_reg.set_index('num','region')

In [32]:
pob_reg

,region,ene2015,feb2015,mar2015,abr2015,may2015,jun2015,jul2015,ago2015,sep2015,...,mar2019,abr2019,may2019,jun2019,jul2019,ago2019,sep2019,oct2019,nov2019,dic2019
num,,,,,,,,,,,,,,,,,,,,,
I,Chalco,1548065,1548065,1548065,1548065,1548065,1548065,1548065,1548065,1548065,...,1647170,1647170,1647170,1647170,1647170,1647170,1647170,1647170,1647170,1647170
II,Cuautitlán,1807861,1807861,1807861,1807861,1807861,1807861,1807861,1807861,1807861,...,1915539,1915539,1915539,1915539,1915539,1915539,1915539,1915539,1915539,1915539
III,Ecatepec de morelos,2500483,2500483,2500483,2500483,2500483,2500483,2500483,2500483,2500483,...,2627164,2627164,2627164,2627164,2627164,2627164,2627164,2627164,2627164,2627164
IV,El oro,358189,358189,358189,358189,358189,358189,358189,358189,358189,...,381308,381308,381308,381308,381308,381308,381308,381308,381308,381308
V,Ixtlahuaca,205545,205545,205545,205545,205545,205545,205545,205545,205545,...,217890,217890,217890,217890,217890,217890,217890,217890,217890,217890
VI,Jilotepec,453854,453854,453854,453854,453854,453854,453854,453854,453854,...,484668,484668,484668,484668,484668,484668,484668,484668,484668,484668
VII,Lerma,272736,272736,272736,272736,272736,272736,272736,272736,272736,...,290306,290306,290306,290306,290306,290306,290306,290306,290306,290306
VIII,Nezahualcóyotl,449005,449005,449005,449005,449005,449005,449005,449005,449005,...,478506,478506,478506,478506,478506,478506,478506,478506,478506,478506
IX,Otumba,2169305,2169305,2169305,2169305,2169305,2169305,2169305,2169305,2169305,...,2295713,2295713,2295713,2295713,2295713,2295713,2295713,2295713,2295713,2295713


In [33]:
xy = edo_mex.select_dtypes(include=['float64'])
yx = pob_reg.select_dtypes(include=['int64'])
abs_edomex = xy

In [34]:
tasas_edomex = ((xy/yx)*100000)

In [35]:
tasas_edomex

,ene2015,feb2015,mar2015,abr2015,may2015,jun2015,jul2015,ago2015,sep2015,oct2015,...,mar2019,abr2019,may2019,jun2019,jul2019,ago2019,sep2019,oct2019,nov2019,dic2019
num,,,,,,,,,,,,,,,,,,,,,
I,1.098145,0.839758,1.227339,1.485726,0.968952,1.485726,1.098145,0.968952,1.291935,1.614919,...,1.335624,1.760595,1.457045,0.728522,1.517755,0.0,0.0,0.0,0.0,0.0
I,0.000000,0.000000,0.000000,0.000000,0.064597,0.064597,0.129194,0.064597,0.000000,0.064597,...,0.000000,0.060710,0.242841,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
I,1.162742,0.452177,0.322984,0.000000,0.258387,0.322984,0.193790,0.387581,0.129194,0.129194,...,0.303551,0.485682,0.485682,0.182131,0.546392,0.0,0.0,0.0,0.0,0.0
I,0.129194,0.193790,0.000000,0.129194,0.000000,0.129194,0.000000,0.000000,0.000000,0.064597,...,0.242841,0.060710,0.242841,0.000000,0.182131,0.0,0.0,0.0,0.0,0.0
I,0.387581,0.710564,0.775161,1.033548,0.581371,0.322984,0.258387,0.581371,0.193790,0.193790,...,1.517755,1.396334,1.274914,1.578465,0.849943,0.0,0.0,0.0,0.0,0.0
I,17.182741,19.379031,17.634918,15.180241,19.056047,13.823709,13.242338,12.402580,16.665967,14.017499,...,29.140890,30.962196,29.505151,30.658645,30.172963,0.0,0.0,0.0,0.0,0.0
I,28.099595,27.453628,25.838708,21.833709,27.647418,22.415079,22.479676,21.316934,24.223789,18.151693,...,21.552117,17.302404,20.034362,19.123709,19.487970,0.0,0.0,0.0,0.0,0.0
I,1.485726,2.777661,2.002500,1.098145,0.645968,0.710564,0.387581,0.581371,1.291935,1.421129,...,4.796105,4.067583,3.581901,3.885452,3.399771,0.0,0.0,0.0,0.0,0.0
I,0.839758,0.645968,1.421129,0.904355,1.421129,0.839758,0.452177,0.581371,0.581371,1.162742,...,5.403207,4.796105,5.038946,4.856815,3.339060,0.0,0.0,0.0,0.0,0.0


In [38]:
tasas_edomex['distrito'] = edo_mex.distrito
tasas_edomex['ordendelitos'] = edo_mex.ordendelitos
tasas_edomex['Delito'] = edo_mex.Delitos



In [39]:
tasas_edomex['S1_2015'] = tasas_edomex.ene2015 + tasas_edomex.feb2015 + tasas_edomex.mar2015 + tasas_edomex.abr2015+  tasas_edomex.may2015 + tasas_edomex.jun2015
tasas_edomex['S2_2015'] = tasas_edomex.jul2015 + tasas_edomex.ago2015 + tasas_edomex.sep2015 + tasas_edomex.oct2015+  tasas_edomex.nov2015 + tasas_edomex.dic2015
tasas_edomex['C1_2015'] = tasas_edomex.ene2015 + tasas_edomex.feb2015 + tasas_edomex.mar2015 + tasas_edomex.abr2015
tasas_edomex['C2_2015'] = tasas_edomex.may2015 + tasas_edomex.jun2015+ tasas_edomex.jul2015 + tasas_edomex.ago2015
tasas_edomex['C3_2015'] = tasas_edomex.sep2015 + tasas_edomex.oct2015+  tasas_edomex.nov2015 + tasas_edomex.dic2015
tasas_edomex['T1_2015'] = tasas_edomex.ene2015 + tasas_edomex.feb2015 + tasas_edomex.mar2015
tasas_edomex['T2_2015'] = tasas_edomex.abr2015+  tasas_edomex.may2015 + tasas_edomex.jun2015
tasas_edomex['T3_2015'] = tasas_edomex.jul2015 + tasas_edomex.ago2015 + tasas_edomex.sep2015
tasas_edomex['T4_2015'] = tasas_edomex.oct2015+  tasas_edomex.nov2015 + tasas_edomex.dic2015
tasas_edomex['B1_2015'] = tasas_edomex.ene2015 + tasas_edomex.feb2015 
tasas_edomex['B2_2015'] = tasas_edomex.mar2015 + tasas_edomex.abr2015
tasas_edomex['B3_2015'] = tasas_edomex.may2015 + tasas_edomex.jun2015
tasas_edomex['B4_2015'] = tasas_edomex.jul2015 + tasas_edomex.ago2015
tasas_edomex['B5_2015'] = tasas_edomex.sep2015 + tasas_edomex.oct2015
tasas_edomex['B6_2015'] = tasas_edomex.nov2015 + tasas_edomex.dic2015
tasas_edomex['A_2015'] = tasas_edomex.ene2015 + tasas_edomex.feb2015 + tasas_edomex.mar2015 + tasas_edomex.abr2015+  tasas_edomex.may2015 + tasas_edomex.jun2015+ tasas_edomex.jul2015 + tasas_edomex.ago2015 + tasas_edomex.sep2015 + tasas_edomex.oct2015+  tasas_edomex.nov2015 + tasas_edomex.dic2015




In [40]:
tasas_edomex['S1_2016'] = tasas_edomex.ene2016 + tasas_edomex.feb2016 + tasas_edomex.mar2016 + tasas_edomex.abr2016+  tasas_edomex.may2016 + tasas_edomex.jun2016
tasas_edomex['S2_2016'] = tasas_edomex.jul2016 + tasas_edomex.ago2016 + tasas_edomex.sep2016 + tasas_edomex.oct2016+  tasas_edomex.nov2016 + tasas_edomex.dic2016
tasas_edomex['C1_2016'] = tasas_edomex.ene2016 + tasas_edomex.feb2016 + tasas_edomex.mar2016 + tasas_edomex.abr2016
tasas_edomex['C2_2016'] = tasas_edomex.may2016 + tasas_edomex.jun2016+ tasas_edomex.jul2016 + tasas_edomex.ago2016
tasas_edomex['C3_2016'] = tasas_edomex.sep2016 + tasas_edomex.oct2016+  tasas_edomex.nov2016 + tasas_edomex.dic2016
tasas_edomex['T1_2016'] = tasas_edomex.ene2016 + tasas_edomex.feb2016 + tasas_edomex.mar2016
tasas_edomex['T2_2016'] = tasas_edomex.abr2016+  tasas_edomex.may2016 + tasas_edomex.jun2016
tasas_edomex['T3_2016'] = tasas_edomex.jul2016 + tasas_edomex.ago2016 + tasas_edomex.sep2016
tasas_edomex['T4_2016'] = tasas_edomex.oct2016+  tasas_edomex.nov2016 + tasas_edomex.dic2016
tasas_edomex['B1_2016'] = tasas_edomex.ene2016 + tasas_edomex.feb2016 
tasas_edomex['B2_2016'] = tasas_edomex.mar2016 + tasas_edomex.abr2016
tasas_edomex['B3_2016'] = tasas_edomex.may2016 + tasas_edomex.jun2016
tasas_edomex['B4_2016'] = tasas_edomex.jul2016 + tasas_edomex.ago2016
tasas_edomex['B5_2016'] = tasas_edomex.sep2016 + tasas_edomex.oct2016
tasas_edomex['B6_2016'] = tasas_edomex.nov2016 + tasas_edomex.dic2016
tasas_edomex['A_2016'] = tasas_edomex.ene2016 + tasas_edomex.feb2016 + tasas_edomex.mar2016 + tasas_edomex.abr2016+  tasas_edomex.may2016 + tasas_edomex.jun2016+ tasas_edomex.jul2016 + tasas_edomex.ago2016 + tasas_edomex.sep2016 + tasas_edomex.oct2016+  tasas_edomex.nov2016 + tasas_edomex.dic2016




In [41]:
tasas_edomex['S1_2017'] = tasas_edomex.ene2017 + tasas_edomex.feb2017 + tasas_edomex.mar2017 + tasas_edomex.abr2017+  tasas_edomex.may2017 + tasas_edomex.jun2017
tasas_edomex['S2_2017'] = tasas_edomex.jul2017 + tasas_edomex.ago2017 + tasas_edomex.sep2017 + tasas_edomex.oct2017+  tasas_edomex.nov2017 + tasas_edomex.dic2017
tasas_edomex['C1_2017'] = tasas_edomex.ene2017 + tasas_edomex.feb2017 + tasas_edomex.mar2017 + tasas_edomex.abr2017
tasas_edomex['C2_2017'] = tasas_edomex.may2017 + tasas_edomex.jun2017+ tasas_edomex.jul2017 + tasas_edomex.ago2017
tasas_edomex['C3_2017'] = tasas_edomex.sep2017 + tasas_edomex.oct2017+  tasas_edomex.nov2017 + tasas_edomex.dic2017
tasas_edomex['T1_2017'] = tasas_edomex.ene2017 + tasas_edomex.feb2017 + tasas_edomex.mar2017
tasas_edomex['T2_2017'] = tasas_edomex.abr2017+  tasas_edomex.may2017 + tasas_edomex.jun2017
tasas_edomex['T3_2017'] = tasas_edomex.jul2017 + tasas_edomex.ago2017 + tasas_edomex.sep2017
tasas_edomex['T4_2017'] = tasas_edomex.oct2017+  tasas_edomex.nov2017 + tasas_edomex.dic2017
tasas_edomex['B1_2017'] = tasas_edomex.ene2017 + tasas_edomex.feb2017 
tasas_edomex['B2_2017'] = tasas_edomex.mar2017 + tasas_edomex.abr2017
tasas_edomex['B3_2017'] = tasas_edomex.may2017 + tasas_edomex.jun2017
tasas_edomex['B4_2017'] = tasas_edomex.jul2017 + tasas_edomex.ago2017
tasas_edomex['B5_2017'] = tasas_edomex.sep2017 + tasas_edomex.oct2017
tasas_edomex['B6_2017'] = tasas_edomex.nov2017 + tasas_edomex.dic2017
tasas_edomex['A_2017'] = tasas_edomex.ene2017 + tasas_edomex.feb2017 + tasas_edomex.mar2017 + tasas_edomex.abr2017+  tasas_edomex.may2017 + tasas_edomex.jun2017+ tasas_edomex.jul2017 + tasas_edomex.ago2017 + tasas_edomex.sep2017 + tasas_edomex.oct2017+  tasas_edomex.nov2017 + tasas_edomex.dic2017




In [42]:
tasas_edomex['S1_2018'] = tasas_edomex.ene2018 + tasas_edomex.feb2018 + tasas_edomex.mar2018 + tasas_edomex.abr2018+  tasas_edomex.may2018 + tasas_edomex.jun2018
tasas_edomex['S2_2018'] = tasas_edomex.jul2018 + tasas_edomex.ago2018 + tasas_edomex.sep2018 + tasas_edomex.oct2018+  tasas_edomex.nov2018 + tasas_edomex.dic2018
tasas_edomex['C1_2018'] = tasas_edomex.ene2018 + tasas_edomex.feb2018 + tasas_edomex.mar2018 + tasas_edomex.abr2018
tasas_edomex['C2_2018'] = tasas_edomex.may2018 + tasas_edomex.jun2018+ tasas_edomex.jul2018 + tasas_edomex.ago2018
tasas_edomex['C3_2018'] = tasas_edomex.sep2018 + tasas_edomex.oct2018+  tasas_edomex.nov2018 + tasas_edomex.dic2018
tasas_edomex['T1_2018'] = tasas_edomex.ene2018 + tasas_edomex.feb2018 + tasas_edomex.mar2018
tasas_edomex['T2_2018'] = tasas_edomex.abr2018+  tasas_edomex.may2018 + tasas_edomex.jun2018
tasas_edomex['T3_2018'] = tasas_edomex.jul2018 + tasas_edomex.ago2018 + tasas_edomex.sep2018
tasas_edomex['T4_2018'] = tasas_edomex.oct2018+  tasas_edomex.nov2018 + tasas_edomex.dic2018
tasas_edomex['B1_2018'] = tasas_edomex.ene2018 + tasas_edomex.feb2018 
tasas_edomex['B2_2018'] = tasas_edomex.mar2018 + tasas_edomex.abr2018
tasas_edomex['B3_2018'] = tasas_edomex.may2018 + tasas_edomex.jun2018
tasas_edomex['B4_2018'] = tasas_edomex.jul2018 + tasas_edomex.ago2018
tasas_edomex['B5_2018'] = tasas_edomex.sep2018 + tasas_edomex.oct2018
tasas_edomex['B6_2018'] = tasas_edomex.nov2018 + tasas_edomex.dic2018
tasas_edomex['A_2018'] = tasas_edomex.ene2018 + tasas_edomex.feb2018 + tasas_edomex.mar2018 + tasas_edomex.abr2018+  tasas_edomex.may2018 + tasas_edomex.jun2018+ tasas_edomex.jul2018 + tasas_edomex.ago2018 + tasas_edomex.sep2018 + tasas_edomex.oct2018+  tasas_edomex.nov2018 + tasas_edomex.dic2018




In [43]:
tasas_edomex['S1_2019'] = tasas_edomex.ene2019 + tasas_edomex.feb2019 + tasas_edomex.mar2019 + tasas_edomex.abr2019+  tasas_edomex.may2019 + tasas_edomex.jun2019
tasas_edomex['S2_2019'] = tasas_edomex.jul2019 + tasas_edomex.ago2019 + tasas_edomex.sep2019 + tasas_edomex.oct2019+  tasas_edomex.nov2019 + tasas_edomex.dic2019
tasas_edomex['C1_2019'] = tasas_edomex.ene2019 + tasas_edomex.feb2019 + tasas_edomex.mar2019 + tasas_edomex.abr2019
tasas_edomex['C2_2019'] = tasas_edomex.may2019 + tasas_edomex.jun2019+ tasas_edomex.jul2019 + tasas_edomex.ago2019
tasas_edomex['C3_2019'] = tasas_edomex.sep2019 + tasas_edomex.oct2019+  tasas_edomex.nov2019 + tasas_edomex.dic2019
tasas_edomex['T1_2019'] = tasas_edomex.ene2019 + tasas_edomex.feb2019 + tasas_edomex.mar2019
tasas_edomex['T2_2019'] = tasas_edomex.abr2019+  tasas_edomex.may2019 + tasas_edomex.jun2019
tasas_edomex['T3_2019'] = tasas_edomex.jul2019 + tasas_edomex.ago2019 + tasas_edomex.sep2019
tasas_edomex['T4_2019'] = tasas_edomex.oct2019+  tasas_edomex.nov2019 + tasas_edomex.dic2019
tasas_edomex['B1_2019'] = tasas_edomex.ene2019 + tasas_edomex.feb2019 
tasas_edomex['B2_2019'] = tasas_edomex.mar2019 + tasas_edomex.abr2019
tasas_edomex['B3_2019'] = tasas_edomex.may2019 + tasas_edomex.jun2019
tasas_edomex['B4_2019'] = tasas_edomex.jul2019 + tasas_edomex.ago2019
tasas_edomex['B5_2019'] = tasas_edomex.sep2019 + tasas_edomex.oct2019
tasas_edomex['B6_2019'] = tasas_edomex.nov2019 + tasas_edomex.dic2019
tasas_edomex['A_2019'] = tasas_edomex.ene2019 + tasas_edomex.feb2019 + tasas_edomex.mar2019 + tasas_edomex.abr2019+  tasas_edomex.may2019 + tasas_edomex.jun2019+ tasas_edomex.jul2019 + tasas_edomex.ago2019 + tasas_edomex.sep2019 + tasas_edomex.oct2019+  tasas_edomex.nov2019 + tasas_edomex.dic2019




In [44]:
tasas_edomex = tasas_edomex.set_index('Delito')

In [45]:
tasas_edomex

,ene2015,feb2015,mar2015,abr2015,may2015,jun2015,jul2015,ago2015,sep2015,oct2015,...,T2_2019,T3_2019,T4_2019,B1_2019,B2_2019,B3_2019,B4_2019,B5_2019,B6_2019,A_2019
Delito,,,,,,,,,,,,,,,,,,,,,
Homicidio doloso,1.098145,0.839758,1.227339,1.485726,0.968952,1.485726,1.098145,0.968952,1.291935,1.614919,...,3.946162,1.517755,0.0,2.671248,3.096220,2.185567,1.517755,0.0,0.0,9.470789
Feminicidio,0.000000,0.000000,0.000000,0.000000,0.064597,0.064597,0.129194,0.064597,0.000000,0.064597,...,0.303551,0.000000,0.0,0.303551,0.060710,0.242841,0.000000,0.0,0.0,0.607102
Homicidio culposo,1.162742,0.452177,0.322984,0.000000,0.258387,0.322984,0.193790,0.387581,0.129194,0.129194,...,1.153494,0.546392,0.0,0.546392,0.789232,0.667812,0.546392,0.0,0.0,2.549828
Secuestro,0.129194,0.193790,0.000000,0.129194,0.000000,0.129194,0.000000,0.000000,0.000000,0.064597,...,0.303551,0.182131,0.0,0.303551,0.303551,0.242841,0.182131,0.0,0.0,1.032073
Extorsión,0.387581,0.710564,0.775161,1.033548,0.581371,0.322984,0.258387,0.581371,0.193790,0.193790,...,4.249713,0.849943,0.0,2.367697,2.914089,2.853379,0.849943,0.0,0.0,8.985108
Robo con violencia,17.182741,19.379031,17.634918,15.180241,19.056047,13.823709,13.242338,12.402580,16.665967,14.017499,...,91.125992,30.172963,0.0,62.956465,60.103086,60.163796,30.172963,0.0,0.0,213.396310
Robo de vehículo,28.099595,27.453628,25.838708,21.833709,27.647418,22.415079,22.479676,21.316934,24.223789,18.151693,...,56.460475,19.487970,0.0,36.304692,38.854520,39.158071,19.487970,0.0,0.0,133.805254
Robo a casa habitación,1.485726,2.777661,2.002500,1.098145,0.645968,0.710564,0.387581,0.581371,1.291935,1.421129,...,11.534936,3.399771,0.0,8.560136,8.863687,7.467353,3.399771,0.0,0.0,28.290948
Robo de negocio,0.839758,0.645968,1.421129,0.904355,1.421129,0.839758,0.452177,0.581371,0.581371,1.162742,...,14.691865,3.339060,0.0,9.835050,10.199312,9.895761,3.339060,0.0,0.0,33.269183


In [46]:
index = tasas_edomex.index

In [47]:
tasas_edomex['RANK']= tasas_edomex.groupby(index)[VF].rank(ascending=0,method='dense')


In [48]:
tasas_edomex['var'] = (tasas_edomex[VF] / tasas_edomex[VI])-1

In [50]:
abs_edomex['distrito'] = edo_mex.distrito
abs_edomex['ordendelitos'] = edo_mex.ordendelitos
abs_edomex['Delitos'] = edo_mex.Delitos


C:\Users\Angel ONC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Angel ONC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [51]:
abs_edomex['S1_2015'] = abs_edomex.ene2015 + abs_edomex.feb2015 + abs_edomex.mar2015 + abs_edomex.abr2015+  abs_edomex.may2015 + abs_edomex.jun2015
abs_edomex['S2_2015'] = abs_edomex.jul2015 + abs_edomex.ago2015 + abs_edomex.sep2015 + abs_edomex.oct2015+  abs_edomex.nov2015 + abs_edomex.dic2015
abs_edomex['C1_2015'] = abs_edomex.ene2015 + abs_edomex.feb2015 + abs_edomex.mar2015 + abs_edomex.abr2015
abs_edomex['C2_2015'] = abs_edomex.may2015 + abs_edomex.jun2015+ abs_edomex.jul2015 + abs_edomex.ago2015
abs_edomex['C3_2015'] = abs_edomex.sep2015 + abs_edomex.oct2015+  abs_edomex.nov2015 + abs_edomex.dic2015
abs_edomex['T1_2015'] = abs_edomex.ene2015 + abs_edomex.feb2015 + abs_edomex.mar2015
abs_edomex['T2_2015'] = abs_edomex.abr2015+  abs_edomex.may2015 + abs_edomex.jun2015
abs_edomex['T3_2015'] = abs_edomex.jul2015 + abs_edomex.ago2015 + abs_edomex.sep2015
abs_edomex['T4_2015'] = abs_edomex.oct2015+  abs_edomex.nov2015 + abs_edomex.dic2015
abs_edomex['B1_2015'] = abs_edomex.ene2015 + abs_edomex.feb2015 
abs_edomex['B2_2015'] = abs_edomex.mar2015 + abs_edomex.abr2015
abs_edomex['B3_2015'] = abs_edomex.may2015 + abs_edomex.jun2015
abs_edomex['B4_2015'] = abs_edomex.jul2015 + abs_edomex.ago2015
abs_edomex['B5_2015'] = abs_edomex.sep2015 + abs_edomex.oct2015
abs_edomex['B6_2015'] = abs_edomex.nov2015 + abs_edomex.dic2015
abs_edomex['A_2015'] = abs_edomex.ene2015 + abs_edomex.feb2015 + abs_edomex.mar2015 + abs_edomex.abr2015+  abs_edomex.may2015 + abs_edomex.jun2015+ abs_edomex.jul2015 + abs_edomex.ago2015 + abs_edomex.sep2015 + abs_edomex.oct2015+  abs_edomex.nov2015 + abs_edomex.dic2015




In [52]:
abs_edomex['S1_2016'] = abs_edomex.ene2016 + abs_edomex.feb2016 + abs_edomex.mar2016 + abs_edomex.abr2016+  abs_edomex.may2016 + abs_edomex.jun2016
abs_edomex['S2_2016'] = abs_edomex.jul2016 + abs_edomex.ago2016 + abs_edomex.sep2016 + abs_edomex.oct2016+  abs_edomex.nov2016 + abs_edomex.dic2016
abs_edomex['C1_2016'] = abs_edomex.ene2016 + abs_edomex.feb2016 + abs_edomex.mar2016 + abs_edomex.abr2016
abs_edomex['C2_2016'] = abs_edomex.may2016 + abs_edomex.jun2016+ abs_edomex.jul2016 + abs_edomex.ago2016
abs_edomex['C3_2016'] = abs_edomex.sep2016 + abs_edomex.oct2016+  abs_edomex.nov2016 + abs_edomex.dic2016
abs_edomex['T1_2016'] = abs_edomex.ene2016 + abs_edomex.feb2016 + abs_edomex.mar2016
abs_edomex['T2_2016'] = abs_edomex.abr2016+  abs_edomex.may2016 + abs_edomex.jun2016
abs_edomex['T3_2016'] = abs_edomex.jul2016 + abs_edomex.ago2016 + abs_edomex.sep2016
abs_edomex['T4_2016'] = abs_edomex.oct2016+  abs_edomex.nov2016 + abs_edomex.dic2016
abs_edomex['B1_2016'] = abs_edomex.ene2016 + abs_edomex.feb2016 
abs_edomex['B2_2016'] = abs_edomex.mar2016 + abs_edomex.abr2016
abs_edomex['B3_2016'] = abs_edomex.may2016 + abs_edomex.jun2016
abs_edomex['B4_2016'] = abs_edomex.jul2016 + abs_edomex.ago2016
abs_edomex['B5_2016'] = abs_edomex.sep2016 + abs_edomex.oct2016
abs_edomex['B6_2016'] = abs_edomex.nov2016 + abs_edomex.dic2016
abs_edomex['A_2016'] = abs_edomex.ene2016 + abs_edomex.feb2016 + abs_edomex.mar2016 + abs_edomex.abr2016+  abs_edomex.may2016 + abs_edomex.jun2016+ abs_edomex.jul2016 + abs_edomex.ago2016 + abs_edomex.sep2016 + abs_edomex.oct2016+  abs_edomex.nov2016 + abs_edomex.dic2016




In [53]:
abs_edomex['S1_2017'] = abs_edomex.ene2017 + abs_edomex.feb2017 + abs_edomex.mar2017 + abs_edomex.abr2017+  abs_edomex.may2017 + abs_edomex.jun2017
abs_edomex['S2_2017'] = abs_edomex.jul2017 + abs_edomex.ago2017 + abs_edomex.sep2017 + abs_edomex.oct2017+  abs_edomex.nov2017 + abs_edomex.dic2017
abs_edomex['C1_2017'] = abs_edomex.ene2017 + abs_edomex.feb2017 + abs_edomex.mar2017 + abs_edomex.abr2017
abs_edomex['C2_2017'] = abs_edomex.may2017 + abs_edomex.jun2017+ abs_edomex.jul2017 + abs_edomex.ago2017
abs_edomex['C3_2017'] = abs_edomex.sep2017 + abs_edomex.oct2017+  abs_edomex.nov2017 + abs_edomex.dic2017
abs_edomex['T1_2017'] = abs_edomex.ene2017 + abs_edomex.feb2017 + abs_edomex.mar2017
abs_edomex['T2_2017'] = abs_edomex.abr2017+  abs_edomex.may2017 + abs_edomex.jun2017
abs_edomex['T3_2017'] = abs_edomex.jul2017 + abs_edomex.ago2017 + abs_edomex.sep2017
abs_edomex['T4_2017'] = abs_edomex.oct2017+  abs_edomex.nov2017 + abs_edomex.dic2017
abs_edomex['B1_2017'] = abs_edomex.ene2017 + abs_edomex.feb2017 
abs_edomex['B2_2017'] = abs_edomex.mar2017 + abs_edomex.abr2017
abs_edomex['B3_2017'] = abs_edomex.may2017 + abs_edomex.jun2017
abs_edomex['B4_2017'] = abs_edomex.jul2017 + abs_edomex.ago2017
abs_edomex['B5_2017'] = abs_edomex.sep2017 + abs_edomex.oct2017
abs_edomex['B6_2017'] = abs_edomex.nov2017 + abs_edomex.dic2017
abs_edomex['A_2017'] = abs_edomex.ene2017 + abs_edomex.feb2017 + abs_edomex.mar2017 + abs_edomex.abr2017+  abs_edomex.may2017 + abs_edomex.jun2017+ abs_edomex.jul2017 + abs_edomex.ago2017 + abs_edomex.sep2017 + abs_edomex.oct2017+  abs_edomex.nov2017 + abs_edomex.dic2017




In [54]:
abs_edomex['S1_2018'] = abs_edomex.ene2018 + abs_edomex.feb2018 + abs_edomex.mar2018 + abs_edomex.abr2018+  abs_edomex.may2018 + abs_edomex.jun2018
abs_edomex['S2_2018'] = abs_edomex.jul2018 + abs_edomex.ago2018 + abs_edomex.sep2018 + abs_edomex.oct2018+  abs_edomex.nov2018 + abs_edomex.dic2018
abs_edomex['C1_2018'] = abs_edomex.ene2018 + abs_edomex.feb2018 + abs_edomex.mar2018 + abs_edomex.abr2018
abs_edomex['C2_2018'] = abs_edomex.may2018 + abs_edomex.jun2018+ abs_edomex.jul2018 + abs_edomex.ago2018
abs_edomex['C3_2018'] = abs_edomex.sep2018 + abs_edomex.oct2018+  abs_edomex.nov2018 + abs_edomex.dic2018
abs_edomex['T1_2018'] = abs_edomex.ene2018 + abs_edomex.feb2018 + abs_edomex.mar2018
abs_edomex['T2_2018'] = abs_edomex.abr2018+  abs_edomex.may2018 + abs_edomex.jun2018
abs_edomex['T3_2018'] = abs_edomex.jul2018 + abs_edomex.ago2018 + abs_edomex.sep2018
abs_edomex['T4_2018'] = abs_edomex.oct2018+  abs_edomex.nov2018 + abs_edomex.dic2018
abs_edomex['B1_2018'] = abs_edomex.ene2018 + abs_edomex.feb2018 
abs_edomex['B2_2018'] = abs_edomex.mar2018 + abs_edomex.abr2018
abs_edomex['B3_2018'] = abs_edomex.may2018 + abs_edomex.jun2018
abs_edomex['B4_2018'] = abs_edomex.jul2018 + abs_edomex.ago2018
abs_edomex['B5_2018'] = abs_edomex.sep2018 + abs_edomex.oct2018
abs_edomex['B6_2018'] = abs_edomex.nov2018 + abs_edomex.dic2018
abs_edomex['A_2018'] = abs_edomex.ene2018 + abs_edomex.feb2018 + abs_edomex.mar2018 + abs_edomex.abr2018+  abs_edomex.may2018 + abs_edomex.jun2018+ abs_edomex.jul2018 + abs_edomex.ago2018 + abs_edomex.sep2018 + abs_edomex.oct2018+  abs_edomex.nov2018 + abs_edomex.dic2018




In [55]:
abs_edomex['S1_2019'] = abs_edomex.ene2019 + abs_edomex.feb2019 + abs_edomex.mar2019 + abs_edomex.abr2019+  abs_edomex.may2019 + abs_edomex.jun2019
abs_edomex['S2_2019'] = abs_edomex.jul2019 + abs_edomex.ago2019 + abs_edomex.sep2019 + abs_edomex.oct2019+  abs_edomex.nov2019 + abs_edomex.dic2019
abs_edomex['C1_2019'] = abs_edomex.ene2019 + abs_edomex.feb2019 + abs_edomex.mar2019 + abs_edomex.abr2019
abs_edomex['C2_2019'] = abs_edomex.may2019 + abs_edomex.jun2019+ abs_edomex.jul2019 + abs_edomex.ago2019
abs_edomex['C3_2019'] = abs_edomex.sep2019 + abs_edomex.oct2019+  abs_edomex.nov2019 + abs_edomex.dic2019
abs_edomex['T1_2019'] = abs_edomex.ene2019 + abs_edomex.feb2019 + abs_edomex.mar2019
abs_edomex['T2_2019'] = abs_edomex.abr2019+  abs_edomex.may2019 + abs_edomex.jun2019
abs_edomex['T3_2019'] = abs_edomex.jul2019 + abs_edomex.ago2019 + abs_edomex.sep2019
abs_edomex['T4_2019'] = abs_edomex.oct2019+  abs_edomex.nov2019 + abs_edomex.dic2019
abs_edomex['B1_2019'] = abs_edomex.ene2019 + abs_edomex.feb2019 
abs_edomex['B2_2019'] = abs_edomex.mar2019 + abs_edomex.abr2019
abs_edomex['B3_2019'] = abs_edomex.may2019 + abs_edomex.jun2019
abs_edomex['B4_2019'] = abs_edomex.jul2019 + abs_edomex.ago2019
abs_edomex['B5_2019'] = abs_edomex.sep2019 + abs_edomex.oct2019
abs_edomex['B6_2019'] = abs_edomex.nov2019 + abs_edomex.dic2019
abs_edomex['A_2019'] = abs_edomex.ene2019 + abs_edomex.feb2019 + abs_edomex.mar2019 + abs_edomex.abr2019+  abs_edomex.may2019 + abs_edomex.jun2019+ abs_edomex.jul2019 + abs_edomex.ago2019 + abs_edomex.sep2019 + abs_edomex.oct2019+  abs_edomex.nov2019 + abs_edomex.dic2019




In [57]:
tasas_edomex = tasas_edomex.reset_index()
tasas_edomex = tasas_edomex.set_index('distrito')

In [58]:
rates_reg_hist_grouped =rates_reg_hist_grouped.reset_index()
rates_reg_hist_grouped =rates_reg_hist_grouped.set_index('distrito')

In [59]:
tasas_edomex.T

distrito,Chalco,Chalco,Chalco,Chalco,Chalco,Chalco,Chalco,Chalco,Chalco,Chalco,...,Zumpango,Zumpango,Zumpango,Zumpango,Zumpango,Zumpango,Zumpango,Zumpango,Zumpango,Zumpango
index,0,1,2,3,4,5,6,7,8,9,...,224,225,226,227,228,229,230,231,232,233
Delito,Homicidio doloso,Feminicidio,Homicidio culposo,Secuestro,Extorsión,Robo con violencia,Robo de vehículo,Robo a casa habitación,Robo de negocio,Robo a transeúnte total,...,Secuestro,Extorsión,Robo con violencia,Robo de vehículo,Robo a casa habitación,Robo de negocio,Robo a transeúnte total,Violación,Trata de personas,Narcomenudeo
ene2015,1.09815,0,1.16274,0.129194,0.387581,17.1827,28.0996,1.48573,0.839758,4.58637,...,0.260308,1.04123,15.6185,20.5643,11.7139,2.08247,9.11079,0.780925,0,15.8788
feb2015,0.839758,0,0.452177,0.19379,0.710564,19.379,27.4536,2.77766,0.645968,7.55782,...,0,0.520616,15.8788,17.701,6.50771,2.34277,6.2474,0.520616,0.260308,10.9329
mar2015,1.22734,0,0.322984,0,0.775161,17.6349,25.8387,2.0025,1.42113,5.36153,...,0.260308,0.520616,14.8376,16.6597,10.9329,5.20616,7.02832,1.82216,0,11.1933
abr2015,1.48573,0,0,0.129194,1.03355,15.1802,21.8337,1.09815,0.904355,5.16774,...,0.260308,0.520616,14.8376,19.2628,15.0979,3.64431,4.94586,1.04123,0,11.9742
may2015,0.968952,0.0645968,0.258387,0,0.581371,19.056,27.6474,0.645968,1.42113,4.39258,...,0,0.260308,10.9329,18.2216,4.42524,2.08247,3.90462,1.56185,0,11.1933
jun2015,1.48573,0.0645968,0.322984,0.129194,0.322984,13.8237,22.4151,0.710564,0.839758,3.10064,...,0,0,13.2757,18.2216,1.30154,2.08247,5.46647,0.260308,0,9.11079
jul2015,1.09815,0.129194,0.19379,0,0.258387,13.2423,22.4797,0.387581,0.452177,3.03605,...,0,0,9.6314,17.1803,5.72678,2.08247,3.90462,1.04123,0,10.4123
ago2015,0.968952,0.0645968,0.387581,0,0.581371,12.4026,21.3169,0.581371,0.581371,2.97145,...,0,0.780925,11.1933,19.5231,5.46647,2.34277,3.38401,0.520616,0,11.4536


In [60]:
crimes = {
    'Robo de negocio':'Robo a negocio'
    
}
tasas_edomex['Delito'] = tasas_edomex.Delito.replace(crimes, regex=True)


In [62]:
rates_hist = pd.merge(tasas_edomex, rates_reg_hist_grouped, how ='inner', on =['Delito', 'distrito']) 

In [63]:
mensual_rates_hist = rates_hist.filter(items=['region', 'Delito',
'ene2011','feb2011','mar2011','abr2011','may2011','jun2011','jul2011','ago2011','sep2011','oct2011''nov2011','dic2011'

'ene2012','feb2012','mar2012','abr2012','may2012','jun2012','jul2012','ago2012','sep2012','oct2012''nov2012','dic2012',

'ene2013','feb2013','mar2013','abr2013','may2013','jun2013','jul2013','ago2013','sep2013','oct2013''nov2013','dic2013',

'ene2014','feb2014','mar2014','abr2014','may2014','jun2014','jul2014','ago2014','sep2014','oct2014''nov2014','dic2014',

'ene2015','feb2015','mar2015','abr2015','may2015','jun2015','jul2015','ago2015','sep2015','oct2015''nov2015','dic2015',

'ene2016','feb2016','mar2016','abr2016','may2016','jun2016','jul2016','ago2016','sep2016','oct2016''nov2016','dic2016',

'ene2017','feb2017','mar2017','abr2017','may2017','jun2017','jul2017','ago2017','sep2017','oct2017''nov2017','dic2017',

'ene2018','feb2018','mar2018','abr2018','may2018','jun2018','jul2018','ago2018','sep2018','oct2018''nov2018','dic2018',

'ene2019','feb2019','mar2019','abr2019','may2019','jun2019','jul2019','ago2019','sep2019','oct2019''nov2019','dic2019'])

In [64]:
mensual_rates_hist.to_csv('files/m_reg_rates.csv')

In [65]:
bi_rates_hist = rates_hist.filter(items=['region', 'Delito',
                                         'B1_2011', 'B2_2011', 'B3_2011', 'B4_2011', 'B5_2011', 'B6_2011',

 'B1_2012', 'B2_2012', 'B3_2012', 'B4_2012', 'B5_2012', 'B6_2012',

 'B1_2013', 'B2_2013', 'B3_2013', 'B4_2013', 'B5_2013', 'B6_2013',

 'B1_2014', 'B2_2014', 'B3_2014', 'B4_2014', 'B5_2014', 'B6_2014',

 'B1_2015', 'B2_2015', 'B3_2015', 'B4_2015', 'B5_2015', 'B6_2015',

 'B1_2016', 'B2_2016', 'B3_2016', 'B4_2016', 'B5_2016', 'B6_2016',

 'B1_2017', 'B2_2017', 'B3_2017', 'B4_2017', 'B5_2017', 'B6_2017',

 'B1_2018', 'B2_2018', 'B3_2018', 'B4_2018', 'B5_2018', 'B6_2018',

 'B1_2019', 'B2_2019', 'B3_2019', 'B4_2019', 'B5_2019', 'B6_2019'])

In [66]:
bi_rates_hist.to_csv('files/bim_reg_rates.csv')

In [67]:
quart_rates_hist = rates_hist.filter(items=['region', 'Delito',
                                        
'T1_2011', 'T2_2011', 'T3_2011', 'T4_2011',

 'T1_2012', 'T2_2012', 'T3_2012', 'T4_2012',

 'T1_2013', 'T2_2013', 'T3_2013', 'T4_2013',

 'T1_2014', 'T2_2014', 'T3_2014', 'T4_2014',

 'T1_2015', 'T2_2015', 'T3_2015', 'T4_2015',

 'T1_2016', 'T2_2016', 'T3_2016', 'T4_2016', 

 'T1_2017', 'T2_2017', 'T3_2017', 'T4_2017',

 'T1_2018', 'T2_2018', 'T3_2018', 'T4_2018',

 'T1_2019', 'T2_2019', 'T3_2019', 'T4_2019'])

In [68]:
quart_rates_hist.to_csv('files/trim_reg_rates.csv')

In [69]:
cuatr_rates_hist = rates_hist.filter(items=['region', 'Delito',
                                        
'C1_2011', 'C2_2011', 'C3_2011',

 'C1_2012', 'C2_2012', 'C3_2012',

 'C1_2013', 'C2_2013', 'C3_2013',

 'C1_2014', 'C2_2014', 'C3_2014',

 'C1_2015', 'C2_2015', 'C3_2015',

 'C1_2016', 'C2_2016', 'C3_2016', 

 'C1_2017', 'C2_2017', 'C3_2017',

 'C1_2018', 'C2_2018', 'C3_2018',

 'C1_2019', 'C2_2019', 'C3_2019'])

In [70]:
cuatr_rates_hist.to_csv('files/cuatr_reg_rates.csv')

In [71]:
bian_rates_hist = rates_hist.filter(items=['region', 'Delito',
                                        
'S1_2011', 'S2_2011',

 'S1_2012', 'S2_2012',

 'S1_2013', 'S2_2013',

 'S1_2014', 'S2_2014',

 'S1_2015', 'S2_2015',

 'S1_2016', 'S2_2016', 

 'S1_2017', 'S2_2017',

 'S1_2018', 'S2_2018',

 'S1_2019', 'S2_2019'])

In [72]:
bian_rates_hist.to_csv('files/sem_reg_rates.csv')

In [73]:
annual_rates_hist = rates_hist.filter(items=['region', 'Delito','A_2011','A_2012','A_2013','A_2014','A_2015','A_2016','A_2017','A_2018','A_2019'])

In [74]:
annual_rates_hist.to_csv('files/anual_reg_rates.csv')

In [75]:
annual_rates_hist

,Delito,A_2011,A_2012,A_2013,A_2014,A_2015,A_2016,A_2017,A_2018,A_2019
distrito,,,,,,,,,,
Chalco,Homicidio doloso,12.888259,14.896470,11.159126,10.509121,13.565322,12.521348,12.013049,13.124710,9.470789
Chalco,Feminicidio,0.000000,0.000000,0.000000,0.000000,0.387581,0.254241,0.437976,0.677802,0.607102
Chalco,Homicidio culposo,7.206338,7.278184,10.691378,13.202083,4.198790,2.478846,2.940694,1.725314,2.549828
Chalco,Secuestro,0.970084,2.312694,1.603707,1.445004,0.775161,1.779684,1.251359,1.232367,1.032073
Chalco,Extorsión,0.000000,11.087327,5.746616,7.093657,5.942903,5.084811,5.130573,10.844831,8.985108
Chalco,Robo con violencia,436.607090,477.367229,314.259692,204.205356,190.495877,137.417030,167.557009,321.524586,213.396310
Chalco,Robo de vehículo,379.510717,449.614902,377.606107,341.546430,280.737566,208.032350,234.254459,295.768113,133.805254
Chalco,Robo a casa habitación,66.658629,60.266082,46.507494,19.179146,15.955402,9.724702,9.322627,44.796545,28.290948
Chalco,Robo a negocio,50.028617,75.298593,37.353002,9.523891,11.821209,7.372977,7.820996,42.947995,33.269183


In [76]:
abs_edomex['Delito'] = abs_edomex.Delitos

In [77]:
abs_edomex = abs_edomex.reset_index()

In [78]:
abs_reg_hist_grouped = abs_reg_hist_grouped.reset_index()

In [79]:
list(abs_edomex)

['num',
 'ene2015',
 'feb2015',
 'mar2015',
 'abr2015',
 'may2015',
 'jun2015',
 'jul2015',
 'ago2015',
 'sep2015',
 'oct2015',
 'nov2015',
 'dic2015',
 'ene2016',
 'feb2016',
 'mar2016',
 'abr2016',
 'may2016',
 'jun2016',
 'jul2016',
 'ago2016',
 'sep2016',
 'oct2016',
 'nov2016',
 'dic2016',
 'ene2017',
 'feb2017',
 'mar2017',
 'abr2017',
 'may2017',
 'jun2017',
 'jul2017',
 'ago2017',
 'sep2017',
 'oct2017',
 'nov2017',
 'dic2017',
 'ene2018',
 'feb2018',
 'mar2018',
 'abr2018',
 'may2018',
 'jun2018',
 'jul2018',
 'ago2018',
 'sep2018',
 'oct2018',
 'nov2018',
 'dic2018',
 'ene2019',
 'feb2019',
 'mar2019',
 'abr2019',
 'may2019',
 'jun2019',
 'jul2019',
 'ago2019',
 'sep2019',
 'oct2019',
 'nov2019',
 'dic2019',
 'distrito',
 'ordendelitos',
 'Delitos',
 'S1_2015',
 'S2_2015',
 'C1_2015',
 'C2_2015',
 'C3_2015',
 'T1_2015',
 'T2_2015',
 'T3_2015',
 'T4_2015',
 'B1_2015',
 'B2_2015',
 'B3_2015',
 'B4_2015',
 'B5_2015',
 'B6_2015',
 'A_2015',
 'S1_2016',
 'S2_2016',
 'C1_2016',
 'C

In [80]:
crimes = {
    'Robo de negocio':'Robo a negocio'
    
}
abs_edomex['Delito'] = abs_edomex.Delito.replace(crimes, regex=True)


In [82]:
abs_hist = pd.merge(abs_edomex, abs_reg_hist_grouped, how ='inner', on =['Delito', 'distrito']) 

In [83]:
abs_hist['S1_2011'] = abs_hist.ene2011 + abs_hist.feb2011 + abs_hist.mar2011 + abs_hist.abr2011+  abs_hist.may2011 + abs_hist.jun2011
abs_hist['S2_2011'] = abs_hist.jul2011 + abs_hist.ago2011 + abs_hist.sep2011 + abs_hist.oct2011+  abs_hist.nov2011 + abs_hist.dic2011
abs_hist['C1_2011'] = abs_hist.ene2011 + abs_hist.feb2011 + abs_hist.mar2011 + abs_hist.abr2011
abs_hist['C2_2011'] = abs_hist.may2011 + abs_hist.jun2011+ abs_hist.jul2011 + abs_hist.ago2011
abs_hist['C3_2011'] = abs_hist.sep2011 + abs_hist.oct2011+  abs_hist.nov2011 + abs_hist.dic2011
abs_hist['T1_2011'] = abs_hist.ene2011 + abs_hist.feb2011 + abs_hist.mar2011
abs_hist['T2_2011'] = abs_hist.abr2011+  abs_hist.may2011 + abs_hist.jun2011
abs_hist['T3_2011'] = abs_hist.jul2011 + abs_hist.ago2011 + abs_hist.sep2011
abs_hist['T4_2011'] = abs_hist.oct2011+  abs_hist.nov2011 + abs_hist.dic2011
abs_hist['B1_2011'] = abs_hist.ene2011 + abs_hist.feb2011 
abs_hist['B2_2011'] = abs_hist.mar2011 + abs_hist.abr2011
abs_hist['B3_2011'] = abs_hist.may2011 + abs_hist.jun2011
abs_hist['B4_2011'] = abs_hist.jul2011 + abs_hist.ago2011
abs_hist['B5_2011'] = abs_hist.sep2011 + abs_hist.oct2011
abs_hist['B6_2011'] = abs_hist.nov2011 + abs_hist.dic2011
abs_hist['A_2011'] = abs_hist.ene2011 + abs_hist.feb2011 + abs_hist.mar2011 + abs_hist.abr2011+  abs_hist.may2011 + abs_hist.jun2011+ abs_hist.jul2011 + abs_hist.ago2011 + abs_hist.sep2011 + abs_hist.oct2011+  abs_hist.nov2011 + abs_hist.dic2011

In [84]:
abs_hist['S1_2012'] = abs_hist.ene2012 + abs_hist.feb2012 + abs_hist.mar2012 + abs_hist.abr2012+  abs_hist.may2012 + abs_hist.jun2012
abs_hist['S2_2012'] = abs_hist.jul2012 + abs_hist.ago2012 + abs_hist.sep2012 + abs_hist.oct2012+  abs_hist.nov2012 + abs_hist.dic2012
abs_hist['C1_2012'] = abs_hist.ene2012 + abs_hist.feb2012 + abs_hist.mar2012 + abs_hist.abr2012
abs_hist['C2_2012'] = abs_hist.may2012 + abs_hist.jun2012+ abs_hist.jul2012 + abs_hist.ago2012
abs_hist['C3_2012'] = abs_hist.sep2012 + abs_hist.oct2012+  abs_hist.nov2012 + abs_hist.dic2012
abs_hist['T1_2012'] = abs_hist.ene2012 + abs_hist.feb2012 + abs_hist.mar2012
abs_hist['T2_2012'] = abs_hist.abr2012+  abs_hist.may2012 + abs_hist.jun2012
abs_hist['T3_2012'] = abs_hist.jul2012 + abs_hist.ago2012 + abs_hist.sep2012
abs_hist['T4_2012'] = abs_hist.oct2012+  abs_hist.nov2012 + abs_hist.dic2012
abs_hist['B1_2012'] = abs_hist.ene2012 + abs_hist.feb2012 
abs_hist['B2_2012'] = abs_hist.mar2012 + abs_hist.abr2012
abs_hist['B3_2012'] = abs_hist.may2012 + abs_hist.jun2012
abs_hist['B4_2012'] = abs_hist.jul2012 + abs_hist.ago2012
abs_hist['B5_2012'] = abs_hist.sep2012 + abs_hist.oct2012
abs_hist['B6_2012'] = abs_hist.nov2012 + abs_hist.dic2012
abs_hist['A_2012'] = abs_hist.ene2012 + abs_hist.feb2012 + abs_hist.mar2012 + abs_hist.abr2012+  abs_hist.may2012 + abs_hist.jun2012+ abs_hist.jul2012 + abs_hist.ago2012 + abs_hist.sep2012 + abs_hist.oct2012+  abs_hist.nov2012 + abs_hist.dic2012

In [85]:
abs_hist['S1_2013'] = abs_hist.ene2013 + abs_hist.feb2013 + abs_hist.mar2013 + abs_hist.abr2013+  abs_hist.may2013 + abs_hist.jun2013
abs_hist['S2_2013'] = abs_hist.jul2013 + abs_hist.ago2013 + abs_hist.sep2013 + abs_hist.oct2013+  abs_hist.nov2013 + abs_hist.dic2013
abs_hist['C1_2013'] = abs_hist.ene2013 + abs_hist.feb2013 + abs_hist.mar2013 + abs_hist.abr2013
abs_hist['C2_2013'] = abs_hist.may2013 + abs_hist.jun2013+ abs_hist.jul2013 + abs_hist.ago2013
abs_hist['C3_2013'] = abs_hist.sep2013 + abs_hist.oct2013+  abs_hist.nov2013 + abs_hist.dic2013
abs_hist['T1_2013'] = abs_hist.ene2013 + abs_hist.feb2013 + abs_hist.mar2013
abs_hist['T2_2013'] = abs_hist.abr2013+  abs_hist.may2013 + abs_hist.jun2013
abs_hist['T3_2013'] = abs_hist.jul2013 + abs_hist.ago2013 + abs_hist.sep2013
abs_hist['T4_2013'] = abs_hist.oct2013+  abs_hist.nov2013 + abs_hist.dic2013
abs_hist['B1_2013'] = abs_hist.ene2013 + abs_hist.feb2013 
abs_hist['B2_2013'] = abs_hist.mar2013 + abs_hist.abr2013
abs_hist['B3_2013'] = abs_hist.may2013 + abs_hist.jun2013
abs_hist['B4_2013'] = abs_hist.jul2013 + abs_hist.ago2013
abs_hist['B5_2013'] = abs_hist.sep2013 + abs_hist.oct2013
abs_hist['B6_2013'] = abs_hist.nov2013 + abs_hist.dic2013
abs_hist['A_2013'] = abs_hist.ene2013 + abs_hist.feb2013 + abs_hist.mar2013 + abs_hist.abr2013+  abs_hist.may2013 + abs_hist.jun2013+ abs_hist.jul2013 + abs_hist.ago2013 + abs_hist.sep2013 + abs_hist.oct2013+  abs_hist.nov2013 + abs_hist.dic2013

In [86]:
abs_hist['S1_2014'] = abs_hist.ene2014 + abs_hist.feb2014 + abs_hist.mar2014 + abs_hist.abr2014+  abs_hist.may2014 + abs_hist.jun2014
abs_hist['S2_2014'] = abs_hist.jul2014 + abs_hist.ago2014 + abs_hist.sep2014 + abs_hist.oct2014+  abs_hist.nov2014 + abs_hist.dic2014
abs_hist['C1_2014'] = abs_hist.ene2014 + abs_hist.feb2014 + abs_hist.mar2014 + abs_hist.abr2014
abs_hist['C2_2014'] = abs_hist.may2014 + abs_hist.jun2014+ abs_hist.jul2014 + abs_hist.ago2014
abs_hist['C3_2014'] = abs_hist.sep2014 + abs_hist.oct2014+  abs_hist.nov2014 + abs_hist.dic2014
abs_hist['T1_2014'] = abs_hist.ene2014 + abs_hist.feb2014 + abs_hist.mar2014
abs_hist['T2_2014'] = abs_hist.abr2014+  abs_hist.may2014 + abs_hist.jun2014
abs_hist['T3_2014'] = abs_hist.jul2014 + abs_hist.ago2014 + abs_hist.sep2014
abs_hist['T4_2014'] = abs_hist.oct2014+  abs_hist.nov2014 + abs_hist.dic2014
abs_hist['B1_2014'] = abs_hist.ene2014 + abs_hist.feb2014 
abs_hist['B2_2014'] = abs_hist.mar2014 + abs_hist.abr2014
abs_hist['B3_2014'] = abs_hist.may2014 + abs_hist.jun2014
abs_hist['B4_2014'] = abs_hist.jul2014 + abs_hist.ago2014
abs_hist['B5_2014'] = abs_hist.sep2014 + abs_hist.oct2014
abs_hist['B6_2014'] = abs_hist.nov2014 + abs_hist.dic2014
abs_hist['A_2014'] = abs_hist.ene2014 + abs_hist.feb2014 + abs_hist.mar2014 + abs_hist.abr2014+  abs_hist.may2014 + abs_hist.jun2014+ abs_hist.jul2014 + abs_hist.ago2014 + abs_hist.sep2014 + abs_hist.oct2014+  abs_hist.nov2014 + abs_hist.dic2014

In [87]:
mensual_abs_hist = abs_hist.filter(items=['region', 'Delito',
'ene2011','feb2011','mar2011','abr2011','may2011','jun2011','jul2011','ago2011','sep2011','oct2011''nov2011','dic2011'

'ene2012','feb2012','mar2012','abr2012','may2012','jun2012','jul2012','ago2012','sep2012','oct2012''nov2012','dic2012',

'ene2013','feb2013','mar2013','abr2013','may2013','jun2013','jul2013','ago2013','sep2013','oct2013''nov2013','dic2013',

'ene2014','feb2014','mar2014','abr2014','may2014','jun2014','jul2014','ago2014','sep2014','oct2014''nov2014','dic2014',

'ene2015','feb2015','mar2015','abr2015','may2015','jun2015','jul2015','ago2015','sep2015','oct2015''nov2015','dic2015',

'ene2016','feb2016','mar2016','abr2016','may2016','jun2016','jul2016','ago2016','sep2016','oct2016''nov2016','dic2016',

'ene2017','feb2017','mar2017','abr2017','may2017','jun2017','jul2017','ago2017','sep2017','oct2017''nov2017','dic2017',

'ene2018','feb2018','mar2018','abr2018','may2018','jun2018','jul2018','ago2018','sep2018','oct2018''nov2018','dic2018',

'ene2019','feb2019','mar2019','abr2019','may2019','jun2019','jul2019','ago2019','sep2019','oct2019''nov2019','dic2019'])

In [88]:
mensual_abs_hist.to_csv('files/m_reg_abs.csv')

In [89]:
bi_abs_hist = abs_hist.filter(items=['region', 'Delito',
                                         'B1_2011', 'B2_2011', 'B3_2011', 'B4_2011', 'B5_2011', 'B6_2011',

 'B1_2012', 'B2_2012', 'B3_2012', 'B4_2012', 'B5_2012', 'B6_2012',

 'B1_2013', 'B2_2013', 'B3_2013', 'B4_2013', 'B5_2013', 'B6_2013',

 'B1_2014', 'B2_2014', 'B3_2014', 'B4_2014', 'B5_2014', 'B6_2014',

 'B1_2015', 'B2_2015', 'B3_2015', 'B4_2015', 'B5_2015', 'B6_2015',

 'B1_2016', 'B2_2016', 'B3_2016', 'B4_2016', 'B5_2016', 'B6_2016',

 'B1_2017', 'B2_2017', 'B3_2017', 'B4_2017', 'B5_2017', 'B6_2017',

 'B1_2018', 'B2_2018', 'B3_2018', 'B4_2018', 'B5_2018', 'B6_2018',

 'B1_2019', 'B2_2019', 'B3_2019', 'B4_2019', 'B5_2019', 'B6_2019'])

In [90]:
bi_abs_hist.to_csv('files/bim_reg_abs.csv')

In [91]:
quart_abs_hist = abs_hist.filter(items=['region', 'Delito',
                                        
'T1_2011', 'T2_2011', 'T3_2011', 'T4_2011',

 'T1_2012', 'T2_2012', 'T3_2012', 'T4_2012',

 'T1_2013', 'T2_2013', 'T3_2013', 'T4_2013',

 'T1_2014', 'T2_2014', 'T3_2014', 'T4_2014',

 'T1_2015', 'T2_2015', 'T3_2015', 'T4_2015',

 'T1_2016', 'T2_2016', 'T3_2016', 'T4_2016', 

 'T1_2017', 'T2_2017', 'T3_2017', 'T4_2017',

 'T1_2018', 'T2_2018', 'T3_2018', 'T4_2018',

 'T1_2019', 'T2_2019', 'T3_2019', 'T4_2019'])

In [92]:
quart_abs_hist.to_csv('files/trim_reg_abs.csv')

In [93]:
cuatr_abs_hist = abs_hist.filter(items=['region', 'Delito',
                                        
'C1_2011', 'C2_2011', 'C3_2011',

 'C1_2012', 'C2_2012', 'C3_2012',

 'C1_2013', 'C2_2013', 'C3_2013',

 'C1_2014', 'C2_2014', 'C3_2014',

 'C1_2015', 'C2_2015', 'C3_2015',

 'C1_2016', 'C2_2016', 'C3_2016', 

 'C1_2017', 'C2_2017', 'C3_2017',

 'C1_2018', 'C2_2018', 'C3_2018',

 'C1_2019', 'C2_2019', 'C3_2019'])

In [94]:
cuatr_abs_hist.to_csv('files/cuatr_reg_abs.csv')

In [95]:
bian_abs_hist = abs_hist.filter(items=['region', 'Delito',
                                        
'S1_2011', 'S2_2011',

 'S1_2012', 'S2_2012',

 'S1_2013', 'S2_2013',

 'S1_2014', 'S2_2014',

 'S1_2015', 'S2_2015',

 'S1_2016', 'S2_2016', 

 'S1_2017', 'S2_2017',

 'S1_2018', 'S2_2018',

 'S1_2019', 'S2_2019'])

In [96]:
bian_abs_hist.to_csv('files/sem_reg_abs.csv')

In [97]:
annual_abs_hist = abs_hist.filter(items=['region', 'Delito','A_2011','A_2012','A_2013','A_2014','A_2015','A_2016','A_2017','A_2018','A_2019'])

In [98]:
tasas_edomex

,index,Delito,ene2015,feb2015,mar2015,abr2015,may2015,jun2015,jul2015,ago2015,...,T4_2019,B1_2019,B2_2019,B3_2019,B4_2019,B5_2019,B6_2019,A_2019,RANK,var
distrito,,,,,,,,,,,,,,,,,,,,,
Chalco,0,Homicidio doloso,1.098145,0.839758,1.227339,1.485726,0.968952,1.485726,1.098145,0.968952,...,0.0,2.671248,3.096220,2.185567,1.517755,0.0,0.0,9.470789,7.0,0.555051
Chalco,1,Feminicidio,0.000000,0.000000,0.000000,0.000000,0.064597,0.064597,0.129194,0.064597,...,0.0,0.303551,0.060710,0.242841,0.000000,0.0,0.0,0.607102,1.0,0.970523
Chalco,2,Homicidio culposo,1.162742,0.452177,0.322984,0.000000,0.258387,0.322984,0.193790,0.387581,...,0.0,0.546392,0.789232,0.667812,0.546392,0.0,0.0,2.549828,13.0,1.322402
Chalco,3,Secuestro,0.129194,0.193790,0.000000,0.129194,0.000000,0.129194,0.000000,0.000000,...,0.0,0.303551,0.303551,0.242841,0.182131,0.0,0.0,1.032073,4.0,0.970523
Chalco,4,Extorsión,0.387581,0.710564,0.775161,1.033548,0.581371,0.322984,0.258387,0.581371,...,0.0,2.367697,2.914089,2.853379,0.849943,0.0,0.0,8.985108,3.0,1.095635
Chalco,5,Robo con violencia,17.182741,19.379031,17.634918,15.180241,19.056047,13.823709,13.242338,12.402580,...,0.0,62.956465,60.103086,60.163796,30.172963,0.0,0.0,213.396310,6.0,0.381747
Chalco,6,Robo de vehículo,28.099595,27.453628,25.838708,21.833709,27.647418,22.415079,22.479676,21.316934,...,0.0,36.304692,38.854520,39.158071,19.487970,0.0,0.0,133.805254,8.0,-0.264666
Chalco,7,Robo a casa habitación,1.485726,2.777661,2.002500,1.098145,0.645968,0.710564,0.387581,0.581371,...,0.0,8.560136,8.863687,7.467353,3.399771,0.0,0.0,28.290948,8.0,0.453083
Chalco,8,Robo a negocio,0.839758,0.645968,1.421129,0.904355,1.421129,0.839758,0.452177,0.581371,...,0.0,9.835050,10.199312,9.895761,3.339060,0.0,0.0,33.269183,7.0,1.926108


In [99]:
tableset = tasas_edomex.filter(items=['Delito','ordendelitos','distrito', VI, VAI, VF,'var','RANK'])

tableset = tableset.sort_values('RANK')

In [100]:
tableset = tableset.sort_values('ordendelitos')

In [101]:
tableset = tableset.round(2)

In [102]:
tableset_piervy = tableset.loc[tableset.RANK == 1.0]

In [103]:
tableset_piervy.sort_values('ordendelitos')

,Delito,ordendelitos,S1_2018,S2_2018,S1_2019,var,RANK
distrito,,,,,,,
Nezahualcóyotl,Homicidio doloso,1,32.67,43.27,39.71,0.22,1.0
Chalco,Feminicidio,2,0.31,0.37,0.61,0.97,1.0
Temascaltepec,Homicidio culposo,3,4.31,6.46,9.56,1.22,1.0
Nezahualcóyotl,Secuestro,4,1.70,1.70,2.72,0.60,1.0
Nezahualcóyotl,Extorsión,5,11.45,21.42,20.06,0.75,1.0
Nezahualcóyotl,Robo con violencia,6,939.69,1035.36,1067.28,0.14,1.0
Nezahualcóyotl,Robo de vehículo,7,601.15,528.18,573.87,-0.05,1.0
Nezahualcóyotl,Robo a casa habitación,8,101.61,118.15,96.55,-0.05,1.0
Nezahualcóyotl,Robo a negocio,9,127.48,194.73,171.99,0.35,1.0


In [104]:
tableset_hd = tableset.loc[tableset.index == 'Homicidio doloso']
tableset_fem = tableset.loc[tableset.index == 'Feminicidio']
tableset_hc = tableset.loc[tableset.index == 'Homicidio culposo']
tableset_ext = tableset.loc[tableset.index == 'Extorsión']
tableset_ext = tableset.loc[tableset.index == 'Secuestro']
tableset_rcv = tableset.loc[tableset.index == 'Robo con violencia']
tableset_rch = tableset.loc[tableset.index == 'Robo a casa habitación']
tableset_rdv = tableset.loc[tableset.index == 'Robo de vehículo']
tableset_rdn = tableset.loc[tableset.index == 'Robo de negocio']
tableset_rt = tableset.loc[tableset.index == 'Robo a transeúnte total']
tableset_vio = tableset.loc[tableset.index == 'Violación']
tableset_tra = tableset.loc[tableset.index == 'Trata de personas']
tableset_nar = tableset.loc[tableset.index == 'Narcomenudeo']


In [112]:
tableset_el_oro = tableset.loc[tableset.index == 'El oro']
tableset_texcoco = tableset.loc[tableset.index == 'Texcoco']
tableset_tlalnepantla = tableset.loc[tableset.index == 'Tlalnepantla']
tableset_temascaltepec = tableset.loc[tableset.index == 'Temascaltepec']
tableset_tenango_del_valle = tableset.loc[tableset.index == 'Tenango del valle']
tableset_ixtlahuaca = tableset.loc[tableset.index == 'Ixtlahuaca']
tableset_lerma = tableset.loc[tableset.index == 'Lerma']
tableset_sultepec = tableset.loc[tableset.index == 'Sultepec']
tableset_toluca = tableset.loc[tableset.index == 'Toluca']
tableset_jilotepec = tableset.loc[tableset.index == 'Jilotepec']
tableset_zumpango = tableset.loc[tableset.index == 'Zumpango']
tableset_tenancingo = tableset.loc[tableset.index == 'Tenancingo']
tableset_ecatepec = tableset.loc[tableset.index == 'Ecatepec de Morelos']
tableset_valle = tableset.loc[tableset.index == 'Valle de bravo']
tableset_cuautitlan = tableset.loc[tableset.index == 'Cuautitlán']
tableset_otumba = tableset.loc[tableset.index == 'Otumba']
tableset_chalco = tableset.loc[tableset.index == 'chalco']
tableset_nezahualcóyotl = tableset.loc[tableset.index == 'Nezahualcóyotl']


In [113]:
tableset_nezahualcóyotl

,Delito,ordendelitos,S1_2018,S2_2018,S1_2019,var,RANK
distrito,,,,,,,
Nezahualcóyotl,Homicidio doloso,1,32.67,43.27,39.71,0.22,1.0
Nezahualcóyotl,Feminicidio,2,1.06,3.18,0.42,-0.61,5.0
Nezahualcóyotl,Homicidio culposo,3,11.03,8.27,9.20,-0.17,2.0
Nezahualcóyotl,Secuestro,4,1.70,1.70,2.72,0.60,1.0
Nezahualcóyotl,Extorsión,5,11.45,21.42,20.06,0.75,1.0
Nezahualcóyotl,Robo con violencia,6,939.69,1035.36,1067.28,0.14,1.0
Nezahualcóyotl,Robo de vehículo,7,601.15,528.18,573.87,-0.05,1.0
Nezahualcóyotl,Robo a casa habitación,8,101.61,118.15,96.55,-0.05,1.0
Nezahualcóyotl,Robo a negocio,9,127.48,194.73,171.99,0.35,1.0


In [ ]:
mensual_rates_hist = rates_hist.filter(items=['region', 'Delito',
'ene2011','feb2011','mar2011','abr2011','may2011','jun2011','jul2011','ago2011','sep2011','oct2011''nov2011','dic2011'

'ene2012','feb2012','mar2012','abr2012','may2012','jun2012','jul2012','ago2012','sep2012','oct2012''nov2012','dic2012',

'ene2013','feb2013','mar2013','abr2013','may2013','jun2013','jul2013','ago2013','sep2013','oct2013''nov2013','dic2013',

'ene2014','feb2014','mar2014','abr2014','may2014','jun2014','jul2014','ago2014','sep2014','oct2014''nov2014','dic2014',

'ene2015','feb2015','mar2015','abr2015','may2015','jun2015','jul2015','ago2015','sep2015','oct2015''nov2015','dic2015',

'ene2016','feb2016','mar2016','abr2016','may2016','jun2016','jul2016','ago2016','sep2016','oct2016''nov2016','dic2016',

'ene2017','feb2017','mar2017','abr2017','may2017','jun2017','jul2017','ago2017','sep2017','oct2017''nov2017','dic2017',

'ene2018','feb2018','mar2018','abr2018','may2018','jun2018','jul2018','ago2018','sep2018','oct2018''nov2018','dic2018',

'ene2019','feb2019','mar2019','abr2019','may2019','jun2019','jul2019','ago2019','sep2019','oct2019''nov2019','dic2019'])

In [114]:
mensual_rates_hist.to_csv('files/m_reg_rates.csv')

In [115]:
bi_rates_hist = rates_hist.filter(items=['region', 'Delito',
                                         'B1_2011', 'B2_2011', 'B3_2011', 'B4_2011', 'B5_2011', 'B6_2011',

 'B1_2012', 'B2_2012', 'B3_2012', 'B4_2012', 'B5_2012', 'B6_2012',

 'B1_2013', 'B2_2013', 'B3_2013', 'B4_2013', 'B5_2013', 'B6_2013',

 'B1_2014', 'B2_2014', 'B3_2014', 'B4_2014', 'B5_2014', 'B6_2014',

 'B1_2015', 'B2_2015', 'B3_2015', 'B4_2015', 'B5_2015', 'B6_2015',

 'B1_2016', 'B2_2016', 'B3_2016', 'B4_2016', 'B5_2016', 'B6_2016',

 'B1_2017', 'B2_2017', 'B3_2017', 'B4_2017', 'B5_2017', 'B6_2017',

 'B1_2018', 'B2_2018', 'B3_2018', 'B4_2018', 'B5_2018', 'B6_2018',

 'B1_2019', 'B2_2019', 'B3_2019', 'B4_2019', 'B5_2019', 'B6_2019'])

In [116]:
bi_rates_hist.to_csv('files/bim_reg_rates.csv')

In [117]:
quart_rates_hist = rates_hist.filter(items=['region', 'Delito',
                                        
'T1_2011', 'T2_2011', 'T3_2011', 'T4_2011',

 'T1_2012', 'T2_2012', 'T3_2012', 'T4_2012',

 'T1_2013', 'T2_2013', 'T3_2013', 'T4_2013',

 'T1_2014', 'T2_2014', 'T3_2014', 'T4_2014',

 'T1_2015', 'T2_2015', 'T3_2015', 'T4_2015',

 'T1_2016', 'T2_2016', 'T3_2016', 'T4_2016', 

 'T1_2017', 'T2_2017', 'T3_2017', 'T4_2017',

 'T1_2018', 'T2_2018', 'T3_2018', 'T4_2018',

 'T1_2019', 'T2_2019', 'T3_2019', 'T4_2019'])

In [118]:
quart_rates_hist.to_csv('files/trim_reg_rates.csv')

In [119]:
cuatr_rates_hist = rates_hist.filter(items=['region', 'Delito',
                                        
'C1_2011', 'C2_2011', 'C3_2011',

 'C1_2012', 'C2_2012', 'C3_2012',

 'C1_2013', 'C2_2013', 'C3_2013',

 'C1_2014', 'C2_2014', 'C3_2014',

 'C1_2015', 'C2_2015', 'C3_2015',

 'C1_2016', 'C2_2016', 'C3_2016', 

 'C1_2017', 'C2_2017', 'C3_2017',

 'C1_2018', 'C2_2018', 'C3_2018',

 'C1_2019', 'C2_2019', 'C3_2019'])

In [120]:
cuatr_rates_hist.to_csv('files/cuatr_reg_rates.csv')

In [121]:
bian_rates_hist = rates_hist.filter(items=['region', 'Delito',
                                        
'S1_2011', 'S2_2011',

 'S1_2012', 'S2_2012',

 'S1_2013', 'S2_2013',

 'S1_2014', 'S2_2014',

 'S1_2015', 'S2_2015',

 'S1_2016', 'S2_2016', 

 'S1_2017', 'S2_2017',

 'S1_2018', 'S2_2018',

 'S1_2019', 'S2_2019'])

In [122]:
bian_rates_hist.to_csv('files/sem_reg_rates.csv')

In [123]:
annual_rates_hist = rates_hist.filter(items=['region', 'Delito','A_2011','A_2012','A_2013','A_2014','A_2015','A_2016','A_2017','A_2018','A_2019'])

In [124]:
victimas_path = r'~\Google Drive\Equipo de datos\playground\Victimas\files\files\absolutos_victimas.csv'